<div class="alert alert-block alert-danger">

# FIT5196 Task 2 in Assessment 1
    
#### Student Name: Hengyu Zhang and Dam Minh Trang Pham
#### Student ID: 31278299 and 30102804

Date: 18/03/2023

Environment: python colab

Libraries used:
* os (for interacting with the operating system, included in Python xxxx)
* pandas 1.1.0 (for dataframe, installed and imported)
* multiprocessing (for performing processes on multi cores, included in Python 3.6.9 package)
* itertools (for performing operations on iterables)
* nltk 3.5 (Natural Language Toolkit, installed and imported)
* nltk.tokenize (for tokenization, installed and imported)
* nltk.stem (for stemming the tokens, installed and imported)

    </div>

<div class="alert alert-block alert-info">
    
## Table of Contents

</div>

[1. Introduction](#Intro) <br>
[2. Importing Libraries](#libs) <br>
[3. Examining Input File](#examine) <br>
[4. Loading and Parsing Files](#load) <br>
$\;\;\;\;$[4.1. Step 1: Data import](#tokenize) <br>
$\;\;\;\;$[4.2. Step 2: Text extraction and cleaning](#whetev) <br>
[5. Writing Output Files](#write) <br>
$\;\;\;\;$[5.1. Step 3: Generate csv file](#write-vocab) <br>
$\;\;\;\;$[5.2. Step 4: Generate the unigram and bigram lists and output as vocab.txt](#write-sparseMat) <br>
$\;\;\;\;$[5.3. Step 5: Generate the sparse numerical representation and output as countvec.txt](#write-txt) <br>
[6. Summary](#summary) <br>
[7. References](#Ref) <br>

<div class="alert alert-block alert-success">
    
## 1.  Introduction  <a class="anchor" name="Intro"></a>

This assessment concerns textual data and the aim is to extract data, process them, and transform them into a proper format. The dataset provided is in the format of a PDF file containing ....

<div class="alert alert-block alert-success">
    
## 2.  Importing Libraries  <a class="anchor" name="libs"></a>

In this assessment, any python packages is permitted to be used. The following packages were used to accomplish the related tasks:



* **os:** to interact with the operating system, e.g., navigate through folders, read, and manage files.

* **re:** to define and use regular expressions for pattern matching in text.

* **pandas:** to work with dataframes for data manipulation and analysis.

* **multiprocessing:** to perform processes on multiple cores for faster performance.

* **nltk:**

  * **Tokenization:** RegexpTokenizer, MWETokenizer for breaking text into words or phrases.
Stemming: PorterStemmer for reducing words to their base or root form.

  * **N-grams:** ngrams and BigramCollocationFinder for finding common word pairs and phrases.

  * **Frequency Analysis:** FreqDist for analyzing the frequency distribution of words.

* **sklearn.feature_extraction.text:** CountVectorizer for converting text data into a numeric matrix of token counts.

* **ast:** to safely evaluate strings containing Python expressions.

* **langdetect:** to detect the language of the text automatically.

* **numpy:** to support large, multi-dimensional arrays and matrices, along with a broad collection of mathematical functions to operate
on these arrays.


In [28]:
import os
import re
import pandas as pd
import multiprocessing
from itertools import chain
import nltk
from nltk.probability import *
from nltk.tokenize import RegexpTokenizer
from nltk.tokenize import MWETokenizer
from nltk.stem import PorterStemmer
from nltk.util import ngrams

from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.probability import FreqDist
from sklearn.feature_extraction.text import CountVectorizer
import ast  # This module helps safely evaluate strings containing Python expressions

from langdetect import detect, DetectorFactory
import numpy as np

from collections import defaultdict
from nltk.probability import FreqDist

-------------------------------------

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<div class="alert alert-block alert-success">
    
## 3.  Examining Input File <a class="anchor" name="examine"></a>



Let's examine what is the content of the file. For this purpose, ....



In [11]:

excel_path = '/content/drive/Shareddrives/FIT5196_S1_2024/A1/Students data/Task 2/Group068.xlsx'

# Load the Excel file
xls = pd.ExcelFile(excel_path)

# Print all sheet names
print("Sheet Names:", xls.sheet_names)
# xls
# xls.sheet_names

Sheet Names: ['Sheet0', 'Sheet1', 'Sheet2', 'Sheet3', 'Sheet4', 'Sheet5', 'Sheet6', 'Sheet7', 'Sheet8', 'Sheet9', 'Sheet10', 'Sheet11', 'Sheet12', 'Sheet13', 'Sheet14', 'Sheet15', 'Sheet16', 'Sheet17', 'Sheet18', 'Sheet19', 'Sheet20', 'Sheet21', 'Sheet22', 'Sheet23', 'Sheet24', 'Sheet25', 'Sheet26', 'Sheet27', 'Sheet28', 'Sheet29']


In [12]:
#some code

# Replace 'Sheet1' with the actual name of a sheet you're interested in
sheet_name = 'Sheet1'
df = pd.read_excel(xls, sheet_name)

# Print column names
print("Column Names:", df.columns.tolist())

# Display the first few rows of the dataframe
print("First few rows of data:")
print(df.head())

Column Names: ['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2']
First few rows of data:
   Unnamed: 0                  Unnamed: 1  \
0         NaN                          id   
1         NaN  UgxrAuSxXXx0VYgTVXV4AaABAg   
2         NaN  Ugw0HXn_jnBUnEjODG54AaABAg   
3         NaN  UgztraKxoUAcFFwKkqd4AaABAg   
4         NaN  UgwFIj52ABEOPbWVXiB4AaABAg   

                                          Unnamed: 2  
0                                            snippet  
1  {'channelId': 'UCet0ZrYmw-V_hsGPb7KsiOQ', 'vid...  
2  {'channelId': 'UCrUTphxarZzISDiXFsFnUUA', 'vid...  
3  {'channelId': 'UCrUTphxarZzISDiXFsFnUUA', 'vid...  
4  {'channelId': 'UCNye-wNBqNL5ZzHSJj3l8Bg', 'vid...  


It is noteiced that excel file contains 29 Sheets, and each sheet has 3 coloumns of data.

<div class="alert alert-block alert-success">
    
## 4.  Loading and Parsing File <a class="anchor" name="load"></a>

<div class="alert alert-block alert-warning">
    
### 4.1. Step 1: Data import <a class="anchor" name="tokenize"></a>

**Question description:**

● Each excel file contains multiple worksheets. Data are positioned differently in each worksheet.

● You are required to combine all data together and remove any duplicates to perform the next step

**Solution Strategy**

In my solution, I begin by loading all the worksheets from a specified Excel file into individual dataframes. I then iterate through each sheet, identifying the row where the 'id' column appears, which I use as the header row. This approach ensures that I accurately capture the start of the relevant data, despite variations in data placement across different sheets. After setting the correct headers and trimming unnecessary rows, I consolidate all the dataframes into a single one. To finalize the preparation, I remove any duplicate rows and empty columns, making the dataset ready for further analysis. This method allows me to efficiently handle and unify data structured differently across multiple sources.

In [13]:
excel_path = '/content/drive/Shareddrives/FIT5196_S1_2024/A1/Students data/Task 2/Group068.xlsx'



# Load all sheets into a single DataFrame
xls = pd.ExcelFile(excel_path)

# Placeholder for all the data frames
all_data_frames = []

for sheet in xls.sheet_names:
    # Read the sheet into a DataFrame
    sheet_df = xls.parse(sheet)

    # Find the header row. Assume 'id' is a unique column name we are looking for
    header_row_idx = None
    for i, row in sheet_df.iterrows():
        if 'id' in row.values:
            header_row_idx = i
            break

    # If 'id' column is found, we process the sheet
    if header_row_idx is not None:
        # Set the header
        sheet_df.columns = sheet_df.iloc[header_row_idx]

        # Skip the rows above the header and reset index
        sheet_df = sheet_df.iloc[header_row_idx + 1:].reset_index(drop=True)

        # Append the DataFrame to the list
        all_data_frames.append(sheet_df)
    else:
        # Handle cases where the header 'id' is not found
        print(f"Header not found in sheet: {sheet}")

# Combine all DataFrames into one
combined_df = pd.concat(all_data_frames, ignore_index=True)


#  df is your DataFrame after reading the Excel file, now drop the duplicate
combined_df = combined_df.dropna(axis=1, how='all').drop_duplicates()







<div class="alert alert-block alert-warning">
    
### 4.2. Step 2: Text extraction and cleaning <a class="anchor" name="whetev"></a>

**Question description:**

You are required to extract the ‘textOriginal’ fields in all top-level comments.
Since the comment data contain emojis, you will need to remove the emojis and normalize the text into lower case for further analysis.
To remove emojis, make sure your text data is in utf-8 format.
The list of emojis to remove are in emoji.txt.
You only require to extract the vocab and countvec lists for English comments from Channels that have at least 15 English comments by using the langdetect library with DetectorFactory.seed = 0. Note you are deciding the language on a comment level, not a sentence level.

**Solution Strategy**

In addressing the task specified in the screenshot, my approach was methodical. I began by extracting the 'textOriginal' field from the 'snippet' data of each comment. To ensure that the text was properly encoded, I made certain that the text data was in UTF-8 format before further processing.

I implemented a function to parse JSON-like strings into dictionaries safely, which allowed me to access and accumulate the 'textOriginal' content and the associated 'channelId' for each comment. This step was essential to extract the necessary information while handling potential parsing errors.

After gathering all 'textOriginal' values, I proceeded to clean the text data. This involved removing any emojis, which I achieved by loading a predefined list of emojis from a file and then filtering them out of the text. I also normalized the text by converting it to lower case, which is crucial for consistent text analysis.

Next, I saved the cleaned text and channel IDs back into the dataframe. To isolate English comments, I set a seed for reproducibility and applied the langdetect library, ensuring that the language detection was consistent and reliable at the comment level. Finally, I filtered the dataframe to retain only those comments that were both identified as English and met the length criteria, preparing it for the subsequent task of extracting vocab and countvec lists, specifically from channels with a minimum number of English comments.

This structured logic ensures that the resulting data is uniform in language and format, facilitating the next steps of analysis.

In [14]:
#use df as a copy of combined_df
df = combined_df

In [15]:


# Placeholder for all 'textOriginal' values
text_originals = []

# Placeholder for all 'channel_id' values
channel_id = []


# Function to safely parse JSON-like strings into a dictionary
def parse_json_string(json_like_string):
    try:
        return ast.literal_eval(json_like_string)
    except Exception as e:
        print(f"Error parsing JSON-like string: {e}")
        return None

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    snippet = row['snippet']  # This is the JSON-like string

    # Parse the JSON-like string into a dictionary
    snippet_dict = parse_json_string(snippet)

    if snippet_dict:
        # Access the 'textOriginal' field under 'topLevelComment'
        text_original = snippet_dict.get('topLevelComment', {}).get('snippet', {}).get('textOriginal', "")
        # Ensure UTF-8 encoding
        text_original = text_original.encode('utf-8').decode('utf-8')

        # Append the text to the list
        text_originals.append(text_original)

        id_channel = snippet_dict.get('channelId', "")

        channel_id.append(id_channel)


# The text_originals variable now contains all the 'textOriginal' texts

In [16]:
emoji_path = '/content/drive/Shareddrives/FIT5196_S1_2024/A1/emoji.txt'

# Function to load emojis from a file
def load_emojis(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        emojis = file.read().splitlines()
    return emojis

# Function to remove emojis from a text
def remove_emojis(text, emojis):
    return ''.join(char for char in text if char not in emojis)

# Load emojis from file
emoji_list = load_emojis(emoji_path)

# Process each text in the list
cleaned_texts = [remove_emojis(text, emoji_list).lower() for text in text_originals]

# Now cleaned_texts contains texts without emojis and in lower case


In [17]:
df['textOriginal'] = cleaned_texts
df['channel_id'] = channel_id
#use df_task4 as a copy of df
df_task4 = df

In [18]:

# Ensure reproducibility in language detection
DetectorFactory.seed = 0


# Remove rows where 'textOriginal' is NaN to ensure language detection runs on valid text
df_task4 = df_task4.dropna(subset=['textOriginal'])

# Function to detect English and check text length
def is_english_and_long(text):
      try:
          return detect(text) == 'en'
      except:
          return False

# Apply the function to determine if the text is English and sufficiently long
df_task4['is_english_and_long'] = df_task4['textOriginal'].apply(is_english_and_long)

#use df_task4_save as a copy of df_task4 which only return trun in 'is_english_and_long'
df_task4_save = df_task4[df_task4['is_english_and_long']]

In [25]:
# Assuming df is your DataFrame loaded as per the given structure

# Function to determine if a channel has at least 15 English comments
def channel_filter(df):
    # Count comments per channel
    counts = df.groupby('channel_id').size()
    # Filter channels with at least 15 comments
    return df[df['channel_id'].isin(counts[counts >= 15].index)]

# Apply the filter
filtered_df = channel_filter(df_task4_save)




<div class="alert alert-block alert-success">
    
    
  
## 5. Writing Output Files <a class="anchor" name="write"></a>

<div class="alert alert-block alert-warning">
    
### 5.1. Step 3: Generate csv file <a class="anchor" name="write-vocab"></a>

**Question description:**

● Generate a csv file that contains unique channel ids along with the counts of top level comments(all language, and english).

● The column names are: channel_id, all_comment_count, eng_comment_count.

**Solution Strategy**


My solution strategy for the task outlined involved several methodical steps to process comment data from various channels. I started by extracting the 'textOriginal' field from each comment's 'snippet' data, making sure to encode the text in UTF-8 format. I used a function to parse JSON-like strings into dictionaries, which allowed me to safely extract the 'textOriginal' content and corresponding 'channelId'.

Following the extraction, I cleaned the data by removing emojis with a preloaded list from 'emoji.txt' and converting the text to lowercase for normalization. After cleaning, I added the processed text and channel IDs back into the dataframe. I then employed the langdetect library to identify English comments, setting a fixed seed for consistent language detection.

Before moving to the final steps of data consolidation and analysis, I implemented a custom function, channel_filter, to determine which channels had at least 15 English comments. This filtering was crucial to ensure that subsequent analysis would only consider channels with a sufficient amount of English comments, as per the task requirements.

After applying the channel filter, I had a dataframe ready for the next phase, which involved analyzing the vocab and countvec lists for these selected channels. My approach not only standardized the data in terms of language but also tailored the dataset to focus on channels with a significant number of English comments, thereby streamlining the analysis process.

With the dataset prepared, I proceeded to the next steps, which involved generating a CSV file with unique channel IDs alongside the count of all comments and English comments, incorporating a custom sorting mechanism for the channel IDs before the final export. This comprehensive approach ensured that the output was aligned with the specific requirements of the task.

In [19]:
# Ensure reproducibility in language detection
DetectorFactory.seed = 0


englsih_comments = []
for comments in cleaned_texts:
  try:
    if detect(comments) == 'en':
      englsih_comments.append("is_english_context!!!")
    else:
      englsih_comments.append("no_english_context!!!")
  except:
    englsih_comments.append("error_context!!!")

df['textOriginal(detect_english)'] = englsih_comments

In [20]:
# Count all comments per channelId
df['all_comment_count'] = df.groupby('channel_id')['channel_id'].transform('count')

# Detect English comments and count per channelId
df['eng_comment'] = df['textOriginal(detect_english)'].apply(lambda x: "is_english_context!!!" in x)
df['eng_comment_count'] = df.groupby('channel_id')['eng_comment'].transform('sum').astype(int)

# Drop duplicates to get unique channel_id with their counts
result_df = df[['channel_id', 'all_comment_count', 'eng_comment_count']].drop_duplicates()

In [21]:

# Now, let's create a custom sorting key based on the specified order "-ABCD...XYZabcd...xyz0123456789"
sort_order = "-0123456789ABCDEFGHIJKLMNOPQRSTUVWXYZ_abcdefghijklmnopqrstuvwxyz"

# We will create a dictionary that maps each character to its corresponding position in the sort order
sort_key = {char: index for index, char in enumerate(sort_order)}

# Now we will create a function that converts a channel_id to a tuple of indexes based on our sort_key.
# This tuple can be used to sort the channel_ids in the desired order.
def sort_key_func(channel_id):
    return tuple(sort_key.get(char, 999) for char in channel_id)  # 999 handles characters not in our sort key

# Apply the sorting function to the channel_id column and sort the DataFrame
result_df['sort_key'] = result_df['channel_id'].apply(sort_key_func)
sorted_result_df = result_df.sort_values(by='sort_key').drop('sort_key', axis=1)

In [22]:
# # Save the result to a CSV file
sorted_result_df.to_csv('/content/drive/MyDrive/5196_results/068_channel_list.csv', index=False)

print("CSV file has been created with channel_id, all_comment_count, and eng_comment_count.")


CSV file has been created with channel_id, all_comment_count, and eng_comment_count.


<div class="alert alert-block alert-warning">
    
### 5.2. Step 4: Generate the unigram and bigram lists and output as vocab.txt <a class="anchor" name="write-sparseMat"></a>

**Question description:**

The following steps must be performed (not necessarily in the same order) to complete the assessment. Please note that the order of preprocessing matters and will result in different vocabulary and hence different count vectors. It is part of the assessment to figure out the correct order of preprocessing which makes the most sense as we learned in the unit. You are encouraged to ask questions and discuss them with the teaching team if in doubt.
a. The word tokenization must use the following regular expression, "[a-zA-Z]+"
b. The context-independent and context-dependent stopwords must be removed
from the vocabulary.
■ For context-independent, The provided context-independent stop
words list (i.e, stopwords_en.txt) must be used.
■ For context-dependent stopwords, you must set the threshold to
words that appear in more than 99% of channels_ids that have at
least 15 english comments.
c. Tokens should be stemmed using the Porter stemmer.
d. Rare tokens must be removed from the vocab (with the threshold set to be
words appear in less than 1% channels_ids that have at least 15 english
comments .
e. Tokens with a length less than 3 should be removed from the vocab.
f. First 200 meaningful bigrams (i.e., collocations) must be included in the
vocab using PMI measure, then makes sure the collocations can be
collocated within the same comment.
g. Calculate the vocabulary containing both unigrams and bigrams.
● Combine the unigrams and bigrams, sort the list alphabetically in an ascending order and output as vocab.txt


**Solution Strategy**

The selected order of data preprocessing steps (aebcbdfg) reflects a strategic approach to refining the raw text data in a way that balances the needs for both simplicity and semantic richness in the resulting vocabulary:

Tokenize (a): Tokenization as the first step is fundamental. It converts raw text into a list of words, which are the basic units for further processing.

Remove Short Tokens (e): I chose to remove short tokens early on because they are often not semantically significant and can be considered 'noise' in the analysis. Doing this early reduces the computational load for subsequent steps.

Remove Context-Independent Stopwords (b - first part): Eliminating these stopwords before stemming ensures that we do not unnecessarily stem words that will be removed anyway. It also reduces the number of tokens to be processed by the stemmer, making the stemming step more efficient.

Stem Tokens (c): Stemming at this point is effective because the tokens are already cleaned of fragments and stopwords. This step simplifies the vocabulary by consolidating different forms of a word into a single token, which is essential for a coherent frequency analysis later on.

Remove Context-Dependent Stopwords (b - second part): After stemming, I can identify context-dependent stopwords more accurately, as the stemming may reveal additional common roots that weren't apparent before.

Remove Rare Tokens (d): Removing rare tokens after stemming and stopword removal ensures that the frequencies are calculated based on a more representative and refined set of tokens, thus eliminating those that are unlikely to be statistically significant.

Detect and Include Meaningful Bigrams (f): Only after cleaning the data thoroughly do I look for meaningful bigrams. This ensures that the bigrams are constructed from the most relevant and frequent tokens, as all irrelevant or rare words have been filtered out.

Final Vocabulary (g): Finally, combining the unigrams and bigrams into one vocabulary after all the filtering ensures that the vocabulary contains the most meaningful and representative words and phrases from the text.

By following this sequence, I ensure that the vocabulary is not only clean but also semantically rich, as it includes significant bigrams alongside the most relevant unigrams. This order aligns with the principles learned in the unit and is designed to yield a more accurate and insightful text analysis outcome.

In [26]:
#use df_filetered as a copy of filtered_df
df_filetered = filtered_df


# Define the tokenization pattern
token_pattern = "[a-zA-Z]+"

# Load context-independent stopwords from a file
with open('/content/drive/Shareddrives/FIT5196_S1_2024/A1/stopwords_en.txt', 'r') as file:
    stop_words_independent = set(file.read().splitlines())

# Initialize the Porter stemmer
stemmer = PorterStemmer()

def tokenize(text):
    # Tokenize using the regex pattern
    tokens = re.findall(token_pattern, str(text))  # Ensure text is string

    # Remove context-independent stopwords
    tokens = [token for token in tokens if token.lower() not in stop_words_independent]

    # Remove short tokens
    tokens = [token for token in tokens if len(token) >= 3]

    # Stem tokens
    tokens = [stemmer.stem(token) for token in tokens]

    return tokens

# Apply tokenization and preprocessing
df_filetered['tokens'] = df_filetered['textOriginal'].apply(tokenize)


<ipython-input-26-9978ef2efa62>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filetered['tokens'] = df_filetered['textOriginal'].apply(tokenize)


In [29]:
# Calculate token presence across channels
channel_token_presence = defaultdict(set)

for index, row in df_filetered.iterrows():
    for token in row['tokens']:
        channel_token_presence[token].add(row['channel_id'])

# Number of channels with at least 15 comments
num_channels = len(set(df_filetered['channel_id']))

# Identify rare tokens (appear in less than 1% of these channels)
rare_tokens = {token for token, channels in channel_token_presence.items() if len(channels) / num_channels < 0.01}

# Identify context-dependent stopwords (appear in more than 99% of these channels)
context_dependent_stopwords = {token for token, channels in channel_token_presence.items() if len(channels) / num_channels > 0.99}

# Filter tokens
all_tokens = sum(df_filetered['tokens'].tolist(), [])
filtered_tokens = [token for token in all_tokens if token not in rare_tokens and token not in context_dependent_stopwords]

# Optionally, create a frequency distribution of the filtered tokens
fdist = FreqDist(filtered_tokens)

# Output a report or use fdist for further analysis
print("Rare Tokens:", rare_tokens)
print("Context-Dependent Stopwords:", context_dependent_stopwords)
print("Most Common Tokens:", fdist.most_common(10))


Rare Tokens: {'manson', 'juici', 'diag', 'sixteen', 'kilmainham', 'dreamhost', 'medley', 'riskier', 'commot', 'dahlia', 'gerechtigkeit', 'interchang', 'cybergrind', 'grammar', 'drawback', 'fascism', 'olu', 'karuna', 'ppmv', 'lamington', 'cotten', 'nauurrr', 'po', 'gramma', 'branko', 'retain', 'stelng', 'creater', 'overwritten', 'ket', 'upanishad', 'omar', 'berg', 'dtc', 'koolaid', 'firewal', 'beni', 'wegotten', 'totoro', 'babyorphan', 'worldview', 'ramp', 'octopi', 'pharao', 'soften', 'suffoc', 'empanada', 'sylvia', 'drunkenli', 'obesess', 'braver', 'phytolith', 'boob', 'cenot', 'sentienc', 'oerson', 'econ', 'kaku', 'premis', 'declan', 'simpul', 'creet', 'flutter', 'xoxoxo', 'ion', 'standardis', 'restless', 'romantic', 'uipath', 'bbg', 'ashnikko', 'apush', 'yaaaay', 'hospic', 'impun', 'contractu', 'msfb', 'rubbl', 'conveyor', 'scammi', 'dha', 'winx', 'promulg', 'transmiss', 'karl', 'xenophiliu', 'shacket', 'guitarcent', 'jay', 'feral', 'incorpor', 'syrian', 'relight', 'sout', 'fingerti

In [30]:
# Bigrams analysis using PMI
bigram_measures = BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(filtered_tokens)
# finder.apply_freq_filter(3)  # Adjust this threshold based on data density
top_bigrams = finder.nbest(bigram_measures.pmi, 200)

# Convert bigrams to a flattened list
bigram_tokens = ['_'.join(bigram) for bigram in top_bigrams]

# Combine, sort unigrams and bigrams
vocab = sorted(set(filtered_tokens + bigram_tokens))

In [31]:
# Output vocabulary to a file with index
with open('/content/drive/MyDrive/5196_results/068_vocab.txt', 'w') as f:
    for index, word in enumerate(vocab):
        f.write(f"{word}:{index}\n")

print("The vocab.txt file with indices has been created.")

The vocab.txt file with indices has been created.


<div class="alert alert-block alert-warning">
    
### 5.3. Step 5: Generate the sparse numerical representation and output as countvec.txt <a class="anchor" name="write-txt"></a>

**Question description:**

1. Re-tokenize your text based on the bigram list generated in step 4 if necessary.
2. Generate sparse representation by using the countvectorizer() function OR directly count the frequency using FreqDist().
3. Mapping the generated token with the vocabs in step 4 if need
4. Output the sparse numerical representation into txt file with the following format:
channel_id1,token1_index:token1_frequency,
token2_index:token2_frequency, token3_index:token3_frequency, ...
channel_id2,token2_index:token2_frequency,
token5_index:token5_frequency, token7_index:token7_frequency, ...
channel_id3,token6_index:token6_frequency,
token9_index:token9_frequency, token12_index:token12_frequency, ...

**Solution Strategy**

In this step, my goal is to convert the preprocessed text into a sparse numerical representation that aligns with the vocabulary generated in the previous step. Here's how my strategy translates into code:

1. **Vocabulary Mapping**: I create a dictionary (`vocab_dict`) that maps each token to a unique index. This dictionary will be used by the `CountVectorizer` to match tokens with their corresponding indices.

2. **Re-Tokenization**: Given that I've included bigrams in the vocabulary, I ensure the tokenizer used by the `CountVectorizer` can recognize these by matching any alphanumeric string or hyphenated tokens, which could represent a bigram.

3. **Sparse Representation**: I use `CountVectorizer`, configured with the custom vocabulary and tokenizer, to transform the `textOriginal` column into a sparse matrix (`X`) representing the frequency of each token.

4. **Dataframe Conversion**: The sparse matrix is converted to a pandas dataframe (`count_df`) to simplify subsequent operations. This dataframe associates each token's frequency with its corresponding channel ID.

5. **Aggregation**: I then aggregate the counts by `channel_id`, summing the frequencies of each token within each channel.

6. **Formatting Output**: For each channel, I iterate through the aggregated counts, formatting each token-frequency pair according to the specified output format and ensuring to filter out zero counts.

7. **File Writing**: Finally, I write these formatted strings to a text file (`068_countvec.txt`), each line representing a channel's data with tokens and their frequencies.

8. **Confirmation**: A print statement confirms the creation and location of the `countvec.txt` file.

By structuring my code this way, I produce a count vector file that is ready for further analysis or input into machine learning models, providing a quantitative representation of the presence of unigrams and bigrams in each channel's comments. This sparse representation is a critical step in text analytics and is aligned with the preprocessing and vocabulary compilation steps that precede it.

In [32]:

# Convert list of vocabulary to a dictionary with token:index
vocab_dict = {v: k for k, v in enumerate(vocab)}

# Initialize CountVectorizer with the vocabulary. The tokenizer regex includes alphanumeric characters and hyphens.
vectorizer = CountVectorizer(vocabulary=vocab_dict, tokenizer=lambda text: re.findall(r'\b\w+\b', text.lower()))

# Tokenize and create the count matrix
X = vectorizer.fit_transform(df_filetered['textOriginal'])

# Convert the matrix to a dataframe to ease the manipulation
count_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Add channel_id to the count dataframe
count_df['channel_id'] = df_filetered['channel_id'].values

# Group by channel_id and sum all other columns (which are token counts)
grouped_counts = count_df.groupby('channel_id').sum()

# Now we need to reformat this into your output specification
# Preparing the output as a list of strings
output = []
for channel_id, row in grouped_counts.iterrows():
    # Extracting tokens and their counts, filtering out zeros
    token_counts = [f"{vocab_dict[token]}:{int(freq)}" for token, freq in row.items() if freq > 0]
    # Joining the tokens and their counts with commas and appending to the output list
    output.append(f"{channel_id}," + ','.join(token_counts))

# Write to a text file. Modify the path as per your environment.
output_path = '/content/drive/MyDrive/5196_results/068_countvec.txt'  # Assuming a Linux environment
with open(output_path, 'w', encoding='utf-8') as file:
    file.writelines("%s\n" % line for line in output)

print(f"The countvec.txt file has been created at {output_path}.")


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


The countvec.txt file has been created at /content/drive/MyDrive/5196_results/068_countvec.txt.


-------------------------------------

<div class="alert alert-block alert-success">
    
## 6. Summary <a class="anchor" name="summary"></a>

To summarize, my solution strategy was to process and analyze text data from multiple channel comments, aiming to create a refined vocabulary and a sparse numerical representation of the text for each channel. Here's how I approached the task:

1. **Data Preparation**: I began by extracting and cleaning the 'textOriginal' field from each comment, ensuring UTF-8 encoding to handle any emojis properly.

2. **Language Filtering**: I identified and isolated English comments using the `langdetect` library, setting a seed for consistent detection and focusing on channels with a substantial number of English comments.

3. **Text Preprocessing**: Following the order of steps aebcbdfg, I tokenized the text, removed short and stop words, applied stemming, and filtered out rare tokens and context-dependent stopwords. This order was carefully chosen based on the nature of the tasks and the principles learned in the unit, ensuring that the most relevant and meaningful tokens were retained.

4. **Bigram Inclusion and Vocabulary Creation**: I then identified meaningful bigrams using the PMI measure and combined them with the remaining unigrams to form the final vocabulary, which was sorted alphabetically and saved as 'vocab.txt'.

5. **Sparse Matrix Generation**: Lastly, I re-tokenized the text to align with the bigrams in the vocabulary and used `CountVectorizer` to generate a sparse matrix of token frequencies. I then aggregated these frequencies by channel ID and formatted the output into a CSV-like text file, following the given specification.

Each step was executed with consideration of the impact on subsequent steps, adhering to the guidelines provided in the assessment. The resulting files, 'vocab.txt' and 'countvec.txt', represent a comprehensive and numerical synthesis of the text data, ready for any further text analysis or machine learning applications. This methodical process ensures both the cleanliness and the analytical richness of the text data extracted from the channels' comments......

-------------------------------------

<div class="alert alert-block alert-success">
    
## 7. References <a class="anchor" name="Ref"></a>

[1] Pandas dataframe.drop_duplicates(), https://www.geeksforgeeks.org/python-pandas-dataframe-drop_duplicates/, Accessed 13/08/2022.



[Link to my workspace] (https://colab.research.google.com/drive/1gAx577hdGqE1225wdZqHchz7Ea8pFRVy?usp=sharing)

## --------------------------------------------------------------------------------------------------------------------------